In [ ]:
import os
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

os.environ['PINECONE_API_KEY'] = '1826dfeb-ad24-4d54-95c5-8b014de6eba1'

# Connect to Pinecone for data access
pc = Pinecone(api_key = os.environ['PINECONE_API_KEY'])
index_name = 'aipi-chatbot'

index = pc.Index(index_name)

# Load in the embeddings model
embeddings_model = SentenceTransformer("WhereIsAI/UAE-Large-V1")
clear_output(wait=True)

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
import torch

# Free CUDA memory
torch.cuda.empty_cache()

ADAPTER_ID = "kahliahogg/mistral-bot"

# GPU/CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Running evaluation on {device}")

# Load model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
    ADAPTER_ID,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map=device,
)

tokenizer = AutoTokenizer.from_pretrained(ADAPTER_ID)

# Load merged model into pipeline
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer
)
clear_output(wait=True)

In [ ]:
import re

Question = "How many electives do I have to take?"

embeddings = embeddings_model.encode(Question)

matches = index.query(
    vector=embeddings.tolist(),
    top_k = 2,
    include_metadata=True
)

DOCUMENTS = matches['matches'][0]['metadata']['text'] + matches['matches'][1]['metadata']['text']

FEW_SHOT_PROMPT = {
    'Q1': "What is the Technical Core of the program? \n Documents: AIPI 510, AIPI 520, AIPI 540, AIPI 561, and AIPI 501",
    'A1': "Yes, the documents are relevant enough to the question",
    'Q2': "Am I able to complete the curriculum without having to do an internship? \n Documents: Internships apply engineering principles to solve one or more problems, define a problsm, and completement material in the AIPI course",
    'A2': "No, the documents are not relevant enough to the question",
}

GRADER_SYSTEM_PROMPT = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    All questions are in relation to the AIPI (Artificial Intelligence for Product Innovation) program. \n
    Do not answer the question itself. Only grade the relevance of the Documents context to the question.

    DOCUMENTS: {context}
    """

BINARY_INCLUSION_PROMPT = "\nAre the provided documents somewhat relevant to the question, answer YES OR NO"

message = [
    {"role": "system", "content": GRADER_SYSTEM_PROMPT},
    {"role": "user", "content": FEW_SHOT_PROMPT['Q1']},
    {"role": "assistant", "content": FEW_SHOT_PROMPT['A1']},
    {"role": "user", "content": FEW_SHOT_PROMPT['Q2']},
    {"role": "assistant", "content": FEW_SHOT_PROMPT['A2']},
    {"role": "system", "content": GRADER_SYSTEM_PROMPT.format(context=matches['matches'][0]['metadata']['text'])},
    {"role": "user", "content": Question+BINARY_INCLUSION_PROMPT},
    {"role": "assistant", "content": ""}
]

prompt = pipe.tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, temperature=0.1, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
response = outputs[0]['generated_text'][len(prompt):].strip()

# Perform regex matching to determine either positive or negative for grader
positive_patterns = [r"\b(?:yes)\b", r"\b(?:are relevant)\b"]
negative_patterns = [r"\b(?:no)\b", r"\b(?:are not relevant)\b"]

positive_regex = re.compile("|".join(positive_patterns), flags=re.IGNORECASE)
negative_regex = re.compile("|".join(negative_patterns), flags=re.IGNORECASE)

is_positive = bool(positive_regex.search(response))

is_negative = bool(negative_regex.search(response))

if is_negative:
    boolean_result = False
elif is_positive:
    boolean_result = True
else:
    boolean_result = True
print(boolean_result)